In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
columns = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magneisum',
           'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
           'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

In [3]:
df = pd.read_csv('data/Wine.csv', names=columns)
df.sample(5)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magneisum,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
167,3,12.82,3.37,2.30,19.5,88,1.48,0.66,0.40,0.97,10.26,0.72,1.75,685
100,2,12.08,2.08,1.70,17.5,97,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710
95,2,12.47,1.52,2.20,19.0,162,2.50,2.27,0.32,3.28,2.60,1.16,2.63,937
33,1,13.76,1.53,2.70,19.5,132,2.95,2.74,0.50,1.35,5.40,1.25,3.00,1235
77,2,11.84,2.89,2.23,18.0,112,1.72,1.32,0.43,0.95,2.65,0.96,2.52,500


In [4]:
df.shape

(178, 14)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magneisum,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [6]:
df.to_csv('data/wine_data.csv', index=False)

In [7]:
features = df.drop('Class', axis=1)
labels = df[['Class']]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

In [9]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()
ytrain_ = torch.from_numpy(y_train.values).long().view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).long().view(1, -1)[0]

In [10]:
Xtrain_.shape, ytrain_.shape, Xtest_.shape, ytest_.shape

(torch.Size([142, 13]),
 torch.Size([142]),
 torch.Size([36, 13]),
 torch.Size([36]))

In [11]:
input_size = 13
output_size = 3
hidden_size = 100

In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    

    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [13]:
model = Net()

In [14]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [15]:
EPOCHS = 1000
for epoch in range(EPOCHS):
    optimizer.zero_grad()
    y_pred = model(Xtrain_)
    loss = loss_fn(y_pred, ytrain_)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item():.4f}')

Epoch: 100, Loss: 0.1277
Epoch: 200, Loss: 0.0396
Epoch: 300, Loss: 0.0332
Epoch: 400, Loss: 0.1359
Epoch: 500, Loss: 0.0325
Epoch: 600, Loss: 0.0292
Epoch: 700, Loss: 0.0272
Epoch: 800, Loss: 0.0256
Epoch: 900, Loss: 0.0244
Epoch: 1000, Loss: 0.0234


In [16]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1233, -0.0938,  0.1221,  ..., -0.1141,  0.2393, -0.0258],
         [-0.2313, -0.0554,  0.1667,  ..., -0.0634,  0.2019,  0.1425],
         [-0.2254,  0.0061, -0.0403,  ...,  0.1570,  0.0203,  0.2269],
         ...,
         [-0.1697, -0.1629, -0.2395,  ..., -0.0481, -0.1474,  0.1120],
         [-0.0280,  0.0753, -0.2378,  ..., -0.0339, -0.0055, -0.1651],
         [ 0.3580, -0.3919, -0.1673,  ...,  1.2716,  0.8647,  0.0970]],
        requires_grad=True),
 Parameter containing:
 tensor([ 7.6994e-02, -1.8227e-01, -1.7633e-01,  7.2131e-02,  6.6214e-02,
         -1.3042e-01, -2.4520e-01,  2.0339e-01,  1.7598e-01,  1.9514e-01,
          8.5368e-02, -1.8526e-01,  6.7546e-01, -2.0807e-01, -1.3042e-01,
         -1.7618e-01,  6.2988e-02,  2.4964e-01, -3.2993e-02,  1.2832e-01,
          5.3011e-01,  1.4783e-01,  2.0502e-01, -4.0730e-02, -1.0399e-01,
         -1.2757e-01, -2.7700e-01, -1.3413e+00,  2.1056e-01, -2.8314e-01,
          7.7584e-02, -1.7615e-01, -2.67

In [17]:
!mkdir models

A subdirectory or file models already exists.


In [18]:
torch.save(model, 'models/wine_classifier.pt')

In [19]:
!ls models

wine_classifier.pt


In [20]:
!cat models/wine_classifier.pt

PK                    archive/data.pklFB ZZZZZZZZZZZZZZ�c__main__
Net
q )�q}q(   trainingq�X   _parametersqccollections
OrderedDict
q)Rq   _buffersqh)RX   _non_persistent_buffers_setq	c__builtin__
set
q
]q�qRq
X   _backward_hooksqh)RqX   _is_full_backward_hookqNX   _forward_hooksqh)RqX   _forward_pre_hooksqh)RqX   _state_dict_hooksqh)RqX   _load_state_dict_pre_hooksqh)Rq   _modulesqh)Rq(X   fc1qctorch.nn.modules.linear
Linear
q)�q}q(h�hh)Rq(X   weightq ctorch._utils
_rebuild_parameter
q!ctorch._utils
_rebuild_tensor_v2
q"((X   storageq#ctorch
FloatStorage
q$X   0q%X   cpuq&Mtq'QK KdK
�q(K
K�q)�h)Rq*tq+Rq,�h)Rq-�q.Rq/X   biasq0h!h"((h#h$X   1q1h&Kdtq2QK Kd�q3K�q4�h)Rq5tq6Rq7�h)Rq8�q9Rq:uhh)Rq;h	h
]q<�q=Rq>hh)Rq?hNhh)Rq@hh)RqAhh)RqBhh)RqChh)RqDX   in_featuresqEK
X   out_featuresqFKdubX   fc2qGh)�qH}qI(h�hh)RqJ(h h!h"((h#h$X   2qKh&M'tqLQK KdKd�qMKdK�qN�h)RqOtqPRqQ�h)RqR�qSRqTh0h!h"((h#h$X   3qUh

In [21]:
new_model = torch.load('models/wine_classifier.pt')

In [22]:
predict_out = new_model(Xtest_)
_, predict_y = torch.max(predict_out, 1)

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f'Accuracy: {accuracy_score(ytest_, predict_y):.4f}')
print(f'Precision: {precision_score(ytest_, predict_y, average="micro"):.4f}')
print(f'Recall: {recall_score(ytest_, predict_y, average="micro"):.4f}')
print(f'F1 Score: {f1_score(ytest_, predict_y, average="micro"):.4f}')

Accuracy: 0.9444
Precision: 0.9444
Recall: 0.9444
F1 Score: 0.9444
